In [75]:
import bs4
import urllib.parse
import csv
import lxml
import re


In [76]:
from urllib.request import urlopen as ureq
from bs4 import BeautifulSoup as soup
from datetime import datetime
from lxml import html, etree

## Fetching the 10Q link

In [77]:
base_url = 'https://www.sec.gov'
cik = "1288776"
acc_num = "000128877614000020"
acc_num_index = acc_num[0:10]+"-"+acc_num[10:12]+"-"+acc_num[12:]+"-index.html"
url_rendered = base_url + "/Archives/edgar/data/" + cik + "/" + acc_num +"/" + acc_num_index
form_url = base_url + "/" + cik + "/" + acc_num
print(url_rendered)

https://www.sec.gov/Archives/edgar/data/1288776/000128877614000020/0001288776-14-000020-index.html


In [78]:
uCLient = ureq(url_rendered)
page_html=uCLient.read()

In [80]:
page_soup = soup(page_html, 'html.parser')
divs = page_soup.find('table',summary="Document Format Files")
url2=divs.find_all('tr')[1].find_all('td')[2].find('a')['href']

## Fetching tables

In [81]:
my_url2=urllib.parse.urljoin(base_url, url2)
my_url2

'https://www.sec.gov/Archives/edgar/data/1288776/000128877614000020/goog2013123110-k.htm'

In [82]:
uCLient2 = ureq(my_url2)
page_html2=uCLient2.read()
page_soup2 = soup(page_html2, 'html.parser')
print(str(page_html2))

b'<DOCUMENT>\n<TYPE>10-K\n<SEQUENCE>1\n<FILENAME>goog2013123110-k.htm\n<DESCRIPTION>FORM 10-K\n<TEXT>\n<!DOCTYPE html PUBLIC "-//W3C//DTD HTML 4.01 Transitional//EN" "http://www.w3.org/TR/html4/loose.dtd">\n<html>\n\t<head>\n\t\t<!-- Document created using WebFilings 1 -->\n\t\t<!-- Copyright 2008-2014 WebFilings LLC. All Rights Reserved -->\n\t\t<title>GOOG 2013.12.31 10-K</title>\n\t</head>\n\t<body style="font-family:Times New Roman;font-size:10pt;">\n<a name="s94205E338A57097DF1D92E71A7D795DE"></a><div></div><br><div style="line-height:120%;padding-top:4px;text-align:center;font-size:14pt;"><font style="font-family:Arial;font-size:14pt;font-weight:bold;">UNITED STATES</font></div><div style="line-height:120%;text-align:center;font-size:14pt;"><font style="font-family:Arial;font-size:14pt;font-weight:bold;">SECURITIES AND EXCHANGE COMMISSION</font></div><div style="line-height:120%;text-align:center;font-size:10pt;"><font style="font-family:Arial;font-size:10pt;font-weight:bold;">Wa

In [83]:
for link in page_soup2.find_all('a'):
    print(link.get_text())
    print(link.get('href'))
    print('end')


None
end

None
end
Table of Contents
#s8F2109C5C45A243A35272E71A82BB851
end
Note About Forward-Looking Statements
#s2FFBB04411B560269CA92E71A85D8A01
end
1
#s2FFBB04411B560269CA92E71A85D8A01
end
Business
#sD639E991EBB55393BBAF2E71A87E5ECB
end
3
#sD639E991EBB55393BBAF2E71A87E5ECB
end
Risk Factors
#s1EBF39222B397626A1572E71A8B0E8ED
end
8
#s1EBF39222B397626A1572E71A8B0E8ED
end
Unresolved Staff Comments
#s749131F2E6B8C60E61D32E71A8D2D8C8
end
20
#s749131F2E6B8C60E61D32E71A8D2D8C8
end
Properties
#s419CE228575E444C182A2E71A90304E5
end
20
#s419CE228575E444C182A2E71A90304E5
end
Legal Proceedings
#sC72117B748D1ECF0FB262E71A9269648
end
21
#sC72117B748D1ECF0FB262E71A9269648
end
Mine Safety Disclosures
#s66C4F05188C1106BA1982E71A957D0AC
end
21
#s66C4F05188C1106BA1982E71A957D0AC
end
Market for Registrant’s Common Equity, Related Stockholder Matters and Issuer Purchases of Equity Securities
#sC557469B021ACAF99D622E71A978C458
end
21
#sC557469B021ACAF99D622E71A978C458
end
Selected Financial Data
#s19D7

In [ ]:
pages=page_soup2.find("a",{'name':'Part1FinInfo'})
page1=pages.parent.parent.parent
print(page1)
table_element = page1.select('div[align="left"]')
table_element

In [ ]:
table_element = page1.select('div[align="left"]')
table_all_rows = []
for table in table_element:
    table_row = table.select('tr')
    for j in table_row:
        table_column = j.select('font')
        row = []
        for k in table_column:
            k_text = k.text.replace(u'\xa0',u'')
            #k_text = k_text.encode("utf-8")
            k_text = str(k_text).strip('b').strip()
            k_text= k_text.strip('"')
            print(k_text)
            row.append(k_text)
        table_all_rows.append(row)
    try:
        with open(('test1.csv'),"a") as my_csv:
            csvWriter = csv.writer(my_csv,delimiter=',')
            try:
                csvWriter.writerows(table_all_rows) 
            except:
                print("no")
    except:
        print("file open error")

In [ ]:
soup_part=page_soup2

In [85]:
def find_between( s, first, last ):
    try:
        start = s.index( first ) + len( first )
        end = s.index( last, start )
        return s[start:end]
    except ValueError:
        return ""

In [86]:
# Working code 
contents=[]
new_soup = find_between( page_soup2.prettify(), '<a name="Part1FinInfo">', '<a name="ConsolidatedFS">' )
print(new_soup)
new_bs = soup(new_soup, 'html.parser')
tables = new_bs.find_all("table")
#print(tables[0])
print(tables)


[]


In [87]:
link_list=[]
link_text=[]

for link in page_soup2.find_all('a'):
    href_link=link.get('href')
    href_text=link.get_text()
#     print(href_link)
    if href_link is not None:
#         print("Heyyy ",href_link)
        href_link=str(href_link).strip('#')
        is_Exists = page_soup2.find("a",{'name':href_link})
#         print(is_Exists)
        if (is_Exists is not None):
            link_list.append(href_link)
            link_text.append(href_text)
print("NEw list",link_list)

NEw list ['s8F2109C5C45A243A35272E71A82BB851', 's2FFBB04411B560269CA92E71A85D8A01', 's2FFBB04411B560269CA92E71A85D8A01', 'sD639E991EBB55393BBAF2E71A87E5ECB', 'sD639E991EBB55393BBAF2E71A87E5ECB', 's1EBF39222B397626A1572E71A8B0E8ED', 's1EBF39222B397626A1572E71A8B0E8ED', 's749131F2E6B8C60E61D32E71A8D2D8C8', 's749131F2E6B8C60E61D32E71A8D2D8C8', 's419CE228575E444C182A2E71A90304E5', 's419CE228575E444C182A2E71A90304E5', 'sC72117B748D1ECF0FB262E71A9269648', 'sC72117B748D1ECF0FB262E71A9269648', 's66C4F05188C1106BA1982E71A957D0AC', 's66C4F05188C1106BA1982E71A957D0AC', 'sC557469B021ACAF99D622E71A978C458', 'sC557469B021ACAF99D622E71A978C458', 's19D7370DAB1A284191372E71A9AA23E1', 's19D7370DAB1A284191372E71A9AA23E1', 'sCF5BD61C9BB5908EEE622E71A9CC8F22', 'sCF5BD61C9BB5908EEE622E71A9CC8F22', 's99C3F70081B44A9133292E71AAA4998F', 's99C3F70081B44A9133292E71AAA4998F', 'sC8DC6507FC6A82D495EE2E71AAC630DB', 'sC8DC6507FC6A82D495EE2E71AAC630DB', 'sB7394C90D0A5736EAE5A2E71AFD92B5F', 'sB7394C90D0A5736EAE5A2E71AF

In [80]:
for item in range(0, (len(link_list)-1)):
    next2 = item + 1
    print(index, link_list[item], link_list[next2])
    

['Part1FinInfo', 'ConsolidatedFS', 'ConsolEarnings', 'ConsolBS', 'ConsolCF', 'NotesToConsolFS', 'Controls', 'OtherInfo', 'LegalProceedings', 'Unregistered', 'Exhibits']
10 Part1FinInfo ConsolidatedFS
10 ConsolidatedFS ConsolEarnings
10 ConsolEarnings ConsolBS
10 ConsolBS ConsolCF
10 ConsolCF NotesToConsolFS
10 NotesToConsolFS Controls
10 Controls OtherInfo
10 OtherInfo LegalProceedings
10 LegalProceedings Unregistered
10 Unregistered Exhibits


In [50]:
for item in range(0, (len(link_list)-1)):
    next2 = item + 1
    item1=link_list[item]
    item2=link_list[next2]
    print("link list elements", item1, item2)
    first = '<a name="' + item1 +'">'
    last = '<a name="' + item2 +'">'
    print("elements",first,last)
    new_soup = find_between( page_soup2.prettify(), first, last )
    new_bs = soup(new_soup, 'html.parser')
    tables = new_bs.find_all("table")
#     print(tables)
    print(len(tables))
        
    table_element=[]
    for table in tables:
         if len(table) > 0:
            table_element.append(table)
    print("lloooaddd",len(table_element))
    #print("Enddddddddddddddddddddddddddddddddddddd")

link list elements Part1FinInfo ConsolidatedFS
elements <a name="Part1FinInfo"> <a name="ConsolidatedFS">
0
link list elements ConsolidatedFS ConsolEarnings
elements <a name="ConsolidatedFS"> <a name="ConsolEarnings">
0
link list elements ConsolEarnings ConsolBS
elements <a name="ConsolEarnings"> <a name="ConsolBS">
2
link list elements ConsolBS ConsolCF
elements <a name="ConsolBS"> <a name="ConsolCF">
2
link list elements ConsolCF NotesToConsolFS
elements <a name="ConsolCF"> <a name="NotesToConsolFS">
0
link list elements NotesToConsolFS Controls
elements <a name="NotesToConsolFS"> <a name="Controls">
109
link list elements Controls OtherInfo
elements <a name="Controls"> <a name="OtherInfo">
0
link list elements OtherInfo LegalProceedings
elements <a name="OtherInfo"> <a name="LegalProceedings">
0
link list elements LegalProceedings Unregistered
elements <a name="LegalProceedings"> <a name="Unregistered">
0
link list elements Unregistered Exhibits
elements <a name="Unregistered"> <a n

In [100]:
for item in range(0, (len(link_list)-1)):
# for item in link_list[:-1]:
    next2 = item + 1
    item1=link_list[item]
    item2=link_list[next2]
#     print("link list elements", item1, item2)
    first = '<a name="' + item1 +'">'
    last = '<a name="' + item2 +'">'
    print("elements",first,last)
    new_soup = find_between( page_soup2.prettify(), first, last )
    new_bs = soup(new_soup, 'html.parser')
    tables = new_bs.find_all("table")
#     print(tables)
  #  print(len(tables))
    if(len(tables)>0):    
        table_all_rows=[]
        for table in tables:
            
            table_row = table.select('tr')
            #print(table_row)
            for j in table_row:
                print(j.find("td")['style'])
#                 print(j.td.attrs('style'))
                
#                 table_column = j.select('font')
#                 row = []
#                 for k in table_column:
#                     k_text = k.text.replace(u'\xa0',u'')
#                     k_text = k_text.replace(u'\n',u'')
#                     k_text = k_text.replace('\n','')
#                     print(k_text)
# #                     k_text = k_text.replace(u'\n',u'')
# #                     k_text = k_text.encode("utf-8")
# #                     k_text = str(k_text).strip('b').strip()
# #                     k_text= k_text.strip('"')
#                     if(k_text!='\n'):
#                         row.append(k_text)
#                 table_all_rows.append(row)
#                 print(len(table_all_rows))
#         try:
#             with open(('./'+item1+'.csv'),"a") as my_csv:
#                 csvWriter = csv.writer(my_csv,delimiter=',')
#                 try:
#                     csvWriter.writerows(table_all_rows) 
#                 except:
#                     print("no")
#         except:
#             print("file open error")

elements <a name="s8F2109C5C45A243A35272E71A82BB851"> <a name="s2FFBB04411B560269CA92E71A85D8A01">


KeyError: 'style'

In [51]:
table_all_rows=[]
for table in table_element:
    table_row = table.select('tr')
    for j in table_row:
        table_column = j.select('font')
        row = []
        for k in table_column:
            k_text = k.text.replace(u'\xa0',u'')
            #k_text = k_text.encode("utf-8")
            k_text = str(k_text).strip('b').strip()
            k_text= k_text.strip('"')
            print(k_text)
            row.append(k_text)
        table_all_rows.append(row)
    try:
        with open(('test1.csv'),"a") as my_csv:
            csvWriter = csv.writer(my_csv,delimiter=',')
            try:
                csvWriter.writerows(table_all_rows) 
            except:
                print("no")
    except:
        print("file open error")







Total Number

Approximate






of Shares

Dollar Value






Purchased as

of Shares that


Total Number

Average

Part of Publicly

May Yet Be


of Shares

Price Paid

Announced

Purchased Under
Period

Purchased

per Share

Program

The Program*













July
  1, 2013 - July 31, 2013

4,263,207

$
194.43


4,263,207

$
6,828,768,788













August
  1, 2013 - August 31, 2013


3,232,800

$
187.25


3,232,800

$
6,223,411,517













September
  1, 2013 - September 30, 2013

3,040,200

$
188.99


3,040,200

$
5,648,842,668













Total


10,536,207

$
190.66


10,536,207





In [16]:
m = re.search(r'<a name =Title</h1>.*?<h1>', html, re.DOTALL)
s = m.start()
e = m.end() - len('<h1>')
target_html = html[s:e]
new_bs = BeautifulSoup(target_html)

TypeError: expected string or bytes-like object

In [ ]:
 #item2=list(next(item))
       # print(type(item2))
        #if str(next(item)) in str(page.attrs):
            #print("It is here", next(item),page.attrs)
        